# **Lecture 3. Optimization and Linear Regression**
## Applied Machine Learning 
**Volodymyr Kuleshov** \
Cornell Tech

## **Part 1: Optimization and Calculaus Background**
In the previous lecture, we have learned what is a supervised machine learning problem. \
Before we turn our anttention to Linear Regression, we will dive deeper into the question of Optimization.

## **Review: Components of A Supervised Machine Learning Problem**
At a high-level, a supervised machine learning problem has the following structure:
$$Dataset + \underbrace{\text{Algorithm}}_\text{Model Class + Objective + Optimizer} \to Predictive ~ Model$$

The predictive model is chosen to model the relationship between the inputs and targets. For instance, it can predict future targets.


## **Optimizer: Notation**
At a high level, an optimizer takes:


*   An objective $J$ (also called a loss function) and
*   A model class $\mathcal{M}$ and finds a model $f \in \mathcal{M}$ with the smallest value of the objective $J$.
$$min_{f \in \mathcal{M}}J(f)$$

Intuitively, this is the function that bests 'fits' the data on the training dataset $\mathcal{D} = \{(x^{(i)},y^{(i)}) \mid i = 1,2, \dots, n\}$.

We will use the quadraric function as our running example for an objective $J$.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12,4]

In [ ]:
def quadratic_function(theta):
  ''' The cost function , J(theta) '''
  return 0.5*(2*theta - 1)**2

We can visualize it

In [ ]:
# First construct a grid of theta1 parameter pairs and their corresponding cost function values
thetas = np.linspace(-0.2,1,10)
f_vals = quadratic_function(thetas[:,np.newaxis])

plt.plot(thetas, f_vals)
plt.xlabel('Theta')
plt.ylabel('Objective value')
plt.title('Simple quadratic function')

## **Calculus Review: Derivatives**
Recall that the derivatives:
$$\frac{df(\theta_0)}{d\theta}$$

of an univariate function $f : \mathbb{R} \to \mathbb{R}$ is the instantaneous rate of change of the function $f(\theta_0)$ with respect to its parameter $\theta$ at the point $\theta_0$.

In [ ]:
def quadratic_derivative(theta):
  return (2*theta - 1)*2

df0 = quadratic_derivative(np.array([[0]])) # derivative at zero
f0 = quadratic_function(np.array([[0]]))
line_length = 0.2

plt.plot(thetas, f_vals)
plt.annotate('', xytext = (0 - line_length, f0 - line_length*df0), xy = (0 + line_length, f0 + line_length*df0),
             arrowprops={'arrowstyle':'-','lw':1.5}, va = 'center', ha='center')
plt.xlabel('Theta')
plt.ylabel('Objective value')
plt.title('Simple quadratic function')

In [ ]:
pts = np.array([[0, 0.5, 0.8]]).reshape((3,1))
df0s = quadratic_derivative(pts)
f0s = quadratic_function(pts)

plt.plot(thetas, f_vals)
for pts, df0s, f0s in zip(pts.flatten(), df0s.flatten(), f0s.flatten()):
  plt.annotate('', xytext = (pts - line_length, f0s - line_length*df0s), xy = (pts + line_length, f0s + line_length*df0s),
             arrowprops={'arrowstyle':'-','lw':1}, va = 'center', ha='center')

plt.xlabel('Theta')
plt.ylabel('Objective value')
plt.title('Simple quadratic function')
  

## **Calculus Review: Partial Derivative**
The partial derivative
$$\frac{\partial f(\theta_0)}{\partial \theta_j}$$
of a multivariate function $f : \mathbb{R}^d \to \mathbb{R}$ is the derivative of $f$ with respect to $\theta_j$ while all other inputs $\theta_k$ for $k \neq i$ are fixed.

## **Calculus Review: The Gradient**
The gradient $\nabla_{\theta}f$ further extends the derivative to multivariate functions $f : \mathbb{R}^d \to \mathbb{R}$, and is defined at the point $\theta_0$ as: 

$$\nabla_{\theta}f(\theta_0) = \begin{bmatrix}
\frac{\partial f(\theta_0)}{\partial \theta_1}\\
\frac{\partial f(\theta_0)}{\partial \theta_2}\\
\vdots\\
\frac{\partial f(\theta_0)}{\partial \theta_j}
\end{bmatrix}.$$

The $j$-th entry of the vector $\nabla_{\theta}f(\theta_0)$ is the partial derivative $\frac{\partial f(\theta_0)}{\partial \theta_j}$ of $f$ with respect to the $j$-th component of $\theta$

We will use a quadratic function as a running example:

In [ ]:
def quadratic_function2d(theta0, theta1):
  
  """ Quadratic objective function J(theta0, theta1).
  
      The inputs theta0 and theta1 are 2-d arrays and we evaluate 
      the objective at each point theta0[i,j] and theta1[i,j].
      We implement in this way so it is easier to plot the level
      curves of the function in 2d.

      Parameters:
      theta0 (np.array): 2d array of first parameter theta0
      theta1 (np.array): 2d array of second parameter theta1

      Returns:
      fvals (np.array): 2d array of objective function values.
      fvals is the same dimension as theta0 and theta1.
      fvals[i,j] is the value at theta0[i,j] and theta1[i,j]. 

      """
  theta0 = np.atleast_2d(np.asarray(theta0))
  theta1 = np.atleast_2d(np.asarray(theta1))
  return 0.5*((2*theta1 - 2)**2 + (theta0 - 3)**2)

Let's visualize this function

In [ ]:
theta0_grid = np.linspace(-4,7,101)
theta1_grid = np.linspace(-1,4,101)
theta_grid = theta0_grid[np.newaxis,:], theta1_grid[:,np.newaxis]
J_grid = quadratic_function2d(theta0_grid[np.newaxis,:], theta1_grid[:,np.newaxis])

X,Y = np.meshgrid(theta0_grid, theta1_grid)
contours = plt.contour(X,Y,J_grid,10)
plt.clabel(contours)
plt.axis('equal')

Let's write down the derivative of the quadratic function

In [ ]:
def quadratic_derivative2d(theta0, theta1):
  """ Derivative of quadratic objective function 

  The input theta0, theta1 are 1d arrays and we evaluate 
  the derivative at each value theta0[i], theta1[i].

  Parameters:
  theta0 (np.array): 1d array of first parameter theta0
  theta1 (np.array): 1d array of second parameter theta1

  Returns:
  grads (np.array): 2d array of partial derivative
  grads is of the same size as theta0 and theta1
  along first dimension and of size 
  two dimension along second dimension
  grads[i,j] is the j-th partial derivative 
  at input theta0[i] and theta1[i].
  """
  # This is the gradient of 0.5*((2*theta1 - 2)**2 + (theta0 - 3)**2)
  grads = np.stack([theta0-2, (2*theta1 - 2)*2], axis = 1)
  grads = grads.reshape([len(theta0),2])
  return grads

We can visualize the derivative

In [ ]:
theta0_pts, theta1_pts = np.array([2.3,-1.35,-2.3]), np.array([2.4,-0.15,2.75])
dfs = quadratic_derivative2d(theta0_pts, theta1_pts)
line_length = 0.2

contours = plt.contour(X, Y, J_grid, 10)
for theta0_pt, theta1_pt, df0 in zip(theta0_pts, theta1_pts, dfs):
  plt.annotate('', xytext=(theta0_pt, theta1_pt),
               xy = (theta0_pt - line_length*df0[0], theta1_pt - line_length*df0[1]),
               arrowprops={'arrowstyle':'->', 'lw':2}, va = 'center', ha = 'center')

plt.scatter(theta0_pts, theta1_pts)
plt.clabel(contours)
plt.xlabel('Theta0')
plt.ylabel('Theta1')
plt.title('Gradient of the Quadratic function')
plt.axis('equal')


## **Part1b: Gradient Descent**
Next, we will use gradients to define an important algorithm called *gradient descent*

### **Calculus Review: The Gradient** 
The gradient $\nabla_{\theta}f$ further extends the derivative to multivariate functions $f : \mathbb{R}^d \to \mathbb{R}$, and is defined at the point $\theta_0$ as: 

$$\nabla_{\theta}f(\theta_0) = \begin{bmatrix}
\frac{\partial f(\theta_0)}{\partial \theta_1}\\
\frac{\partial f(\theta_0)}{\partial \theta_2}\\
\vdots\\
\frac{\partial f(\theta_0)}{\partial \theta_j}
\end{bmatrix}.$$

The $j$-th entry of the vector $\nabla_{\theta}f(\theta_0)$ is the partial derivative $\frac{\partial f(\theta_0)}{\partial \theta_j}$ of $f$ with respect to the $j$-th component of $\theta$

### **Gradient Descent: Intuition**
Gradient descent is a very common optimization algorithm used in machine learning.\
The intuition behind the gradient descent is to repeatedly obtain the gradient to determine the direction in which the function decrease most steeply and take a step in that direction.

### **Gradient Descent: Notation**
More formally, if we want to optimize $J(\theta)$, we start with an initial guess $\theta_0$ for the parameters and repeat the following update until $\theta$ is no longer changing:
$$\theta_i := \theta_{i-1} - \alpha.\nabla_{\theta}J(\theta_{i-1}) $$ 

As code, this method may look as follows:
```python
theta, theta_prev = random_initialization()
while norm(theta - theta_prev) > convergence_threshold:
    theta_prev = theta
    theta = theta_prev - step_size * gradient(theta_prev)
```
In the above algorithm, we stop when $||\theta_i - \theta_{i-1}||$ is small.

Implementing in numpy

In [ ]:
convergence_threshold = 1e-1
step_size = 2e-1
theta, theta_prev = np.array([[-2],[3]]), np.array([[0],[0]])
opt_pts = [theta.flatten()]
opt_grads = []

while np.linalg.norm(theta - theta_prev) > convergence_threshold:
  # We repeat this while the value of the function is decreasing
  theta_prev = theta
  gradient = quadratic_derivative2d(*theta).reshape([2,1])
  theta = theta_prev - step_size*gradient
  opt_pts += [theta.flatten()]
  opt_grads += [gradient.flatten()] 

We can now visualize the gradient descent

In [ ]:
opt_pts = np.array(opt_pts)
opt_grads = np.array(opt_grads)

contours = plt.contour(X, Y, J_grid, 10)
plt.clabel(contours)
plt.scatter(opt_pts[:,1], opt_pts[:,0])

for opt_pt, opt_grad in zip(opt_pts, opt_grads):
  plt.annotate('', xytext=(opt_pt[0], opt_pt[1]),
               xy = (opt_pt[0] - 0.8*step_size*opt_grad[0], opt_pt[1] - 0.8*step_size*opt_grad[1]),
               arrowprops = {'arrowstyle':'->','lw':2 }, va = 'center', ha = 'center')
  
plt.axis('equal')

## **Part 2: Gradient Descent in Linear Model**
Let's now use gradient descent to derive a supervised learning algorithm for linear models.


### **Review: Linear Model family**
Recall that a linear model has the form
$$y = \theta_0 + \theta_1x_1 + \theta_2x_2 + \dots + \theta_dx_d$$ 

where $x \in \mathbb{R}^d$ is a vector of features and $y$ is the target. The $\theta_j$ is the parameter of the model.

By using the notation $x_0 = 1$, we can represent the model in a vectorized form:
$$f_{\theta}(x) = \sum_{j=0}^d\theta_jx_j = \theta^\top x$$

Let's define our model in Python

In [ ]:
def f(X,theta):
  """ The linear model we are trying to fit.

  Parameters:
  theta (np.array): d-dimensional vector of parameters
  X (np.parameters): (n,d)-dimensional data matrix

  Returns:
  y_pred (np.array): n-dimensional vector of predicted target 
  """
  return X.dot(theta)

### **An Objective: Mean Squared Error**
We pick $\theta$ to minimize the mean squared error (MSE). Slight variants of this objective are also known as the residual sum of squares (RSS) or the sum of squares residual (SSR).
$$J(\theta) = \frac{1}{2n}\sum_{i=1}^{n}(y^{(i)} - \theta^\top x^{(i)})^2$$
In other words, we are looking for the best compromise in $\theta$ over all the data pointd.

Let's implement the MSE.

In [ ]:
def mean_squared_error(theta, X, y):
  return 0.5*np.mean((y - f(X,theta))**2)

### **Mean Squared Error: Partial Derivatives**
Let's work out what a partial derivative is for MSE loss for a linear model.

$$ \begin{align*}
\frac{\partial J(\theta)}{\partial \theta_j} &= \frac{\partial}{\partial \theta_j}\frac{1}{2}(f_{\theta}(x) - y)^2 \\
&= (f_{\theta}(x) - y)\frac{\partial}{\partial \theta_j}(f_{\theta}(x) - y) \\
&= (f_{\theta}(x) - y)\frac{\partial}{\partial \theta_j}(\sum_{k=0}^d\theta_kx_k -y)\\
&= (f_{\theta}(x) - y).x_j
\end{align*}$$

### **Mean Squared Error: The Gradient**
We can use this derivative to obtain the gradient for MSE for a linear model.
$$\begin{align*}
\nabla_{\theta}J(\theta)  = \begin{bmatrix}
\frac{\partial J(\theta)}{\theta_0}\\
\frac{\partial J(\theta)}{\theta_1}\\
\vdots\\
\frac{\partial J(\theta)}{\theta_d}
\end{bmatrix}
= \begin{bmatrix}
(f_{\theta}(x) - y).x_0 \\
(f_{\theta}(x) - y).x_1 \\
\vdots \\
(f_{\theta}(x) - y).x_d
\end{bmatrix}.
= (f_{\theta}(x) - y)\cdot\bf{x}.
\end{align*}$$

Let's implement the gradient

In [ ]:
def mse_gradient(theta, X, y):
  """ Returns: 
  grads (np.array): d-dimensional gradient of the MSe 
  """
  return np.mean((f(X,theta) - y)*X.T, axis=1)

### **EXAMPLE: The UCI Diabetes Dataset**
In this section, we are going to again use the UCI Diabetes dataset.


*   For each patient, we have a acess to a measurement of their body mass index (bmi) and a quantative diabetes risk score (0-400).
*   We are interested in understanding how the BMI  affects the individual's diabetes risk.



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8,4]

import numpy as np
import pandas as pd
from sklearn import datasets

# Load the diabetes dataset
X, y = datasets.load_diabetes(return_X_y=True, as_frame=True)

# Add an extra column of ones
X['one'] = 1

# Collect 20 data point and only use bmi dimension
X_train = X.iloc[-20:].loc[:,['bmi','one']]
y_train = y.iloc[-20:]/300

plt.scatter(X_train.loc[:,['bmi']], y_train, color='black')
plt.xlabel('Body Mass Index (BMI)')
plt.ylabel('Diabetes Risk')

### **Gradient Descent for Linear Regression**
Putting this together with the gradient dascent algorithm, we obtain a method for training linear model.
```python
theta, theta_prev = random_initialization()
while abs(J(theta) - J(theta_prev)) > conv_threshold:
    theta_prev = theta
    theta = theta_prev - step_size * (f(x, theta)-y) * x
```
This update also known as Least Mean Square (LMS) or Widrow Hoff learning rule.

In [ ]:
threshold = 1e-6
step_size = 4e-1
theta, theta_prev = np.array([2,1]), np.ones(2,)
opt_pts = [theta]
opt_grads = []
iter = 0

while abs(mean_squared_error(theta, X_train, y_train) - mean_squared_error(theta_prev, X_train, y_train)) > threshold:
  if iter % 100 == 0:
    print('Iteration %d. MSE: %6f' %(iter, mean_squared_error(theta, X_train, y_train)))
  theta_prev = theta
  gradient = mse_gradient(theta, X_train, y_train)
  theta = theta_prev - step_size*gradient
  opt_pts += [theta]
  opt_grads += [gradient]
  iter += 1



In [ ]:
x_line = np.stack([np.linspace(-0.1, 0.1, 10), np.ones(10,)])
y_line = opt_pts[-1].dot(x_line)

plt.scatter(X_train.loc[:,['bmi']], y_train, color='black')
plt.plot(x_line[0], y_line)
plt.xlabel('Body Mass Index (BMI)')
plt.ylabel('Diabetes Risk')

## **Part 3: Ordinary Least Square**
In practice, there is more effective way than gradient descent to find linear  model parameters.\
We will see this method here, which will lead us to our first non-toy algorithm: Ordinary Least Squares.


### **Review: The Gradient**
The gradient $\nabla_{\theta}f$ further extends the derivative to multivariate functions $f : \mathbb{R}^d \to \mathbb{R}$, and is defined at a point $\theta_0$ as:
$$\nabla_{\theta}f(\theta_0) = \begin{bmatrix}
\frac{\partial f(\theta_0)}{\partial \theta_1}\\
\frac{\partial f(\theta_0)}{\partial \theta_2}\\
\vdots\\
\frac{\partial f(\theta_0)}{\partial \theta_d}\\
\end{bmatrix}$$

In other words, the $j$-th entry of the vector $\nabla_{\theta}f(\theta_0)$ is the partial derivative $\frac{\partial f(\theta_0)}{\partial \theta_j}$ of $f$ with respect to the $j$-th component of $\theta$

### **The UCI Diabetes Dataset**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8,4]

import numpy as np
import pandas as pd
from sklearn import datasets

# Load the diabetes dataset
X, y = datasets.load_diabetes(return_X_y=True, as_frame=True)

# Add an extra columns of ones
X['ones'] = 1


# Collect 20 data point and only use bmi dimension
X_train = X.iloc[-20:]
y_train = y.iloc[-20:]

plt.scatter(X_train.loc[:,['bmi']], y_train, color='black')
plt.xlabel('Body Mass Index (BMI)')
plt.ylabel('Diabetes Risk')

### **Notation: Design Matrix**
Machine Learning algorithms are most easily defined in the language of linear algebra. Therefore, it will be useful to represent the entire dataset as one matrix $X \in \mathbb{R}^{n \times d}$, of the form:
$$X = \begin{bmatrix}
x_1^{(1)} &x_2^{(1)} \ldots &x_d^{(1)} \\
x_1^{(2)} &x_2^{(2)} \ldots &x_d^{(2)} \\
\vdots \\
x_1^{(n)} &x_2^{(n)} \ldots &x_d^{(n)} \\
\end{bmatrix}
= \begin{bmatrix}
- & (x^{(1)})^\top & - \\
- & (x^{(2)})^\top & - \\
& \vdots & \\
- & (x^{(n)})^\top & - \\
\end{bmatrix}$$

We can view the design matrix for the diabetes dataset

In [ ]:
X_train.head()

### **Notation: Design Matrix**
Similarly, we can vectorize the target variables into a vector $y \in \mathbb{R}^n$ of the form:
$$y = \begin{bmatrix}
y^{(1)}\\
y^{(2)}\\
\vdots \\
y^{(n)}
\end{bmatrix}.$$

### **Squared Error in Matrix Form**
Recall that we may fit a linear model by choosing $\theta$ that minimize the squared error:
$$J(\theta) = \frac{1}{2}\sum_{i=1}^n(y^{(i)} - \theta^\top x^{(i)})^2$$

In other words, we are looking for the most compromise in $\theta$ over all the data points. \\

We can write this sum in vector-form as:
$$J(\theta) = \frac{1}{2}(y - X\theta)^\top(y - X\theta) = \frac{1}{2}\|y - X\theta\|^2.$$
where $X$ is the design matrix and $\|\cdot\|$ denotes the Euclidean norm.

### **The Gradient of the Squared Error**
We have a gradient for the mean squared error as follows:

$$ \begin{align*}
\nabla_{\theta}J(\theta) &= \nabla_{\theta}\frac{1}{2}(X\theta - y)^\top(X\theta - y)\\
 &= \frac{1}{2}\nabla_{\theta}((X\theta)^\top(X\theta) - (X\theta)^\top y - y^\top (X\theta) +y^\top y) \\
 &= \frac{1}{2}\nabla_{\theta}(\theta^\top (X^\top X)\theta - 2(X\theta)^\top y)\\
 &= \frac{1}{2}(2(X^\top X)\theta - 2X^\top y)\\
 &= (X^\top X)\theta - X^\top y.
\end{align*}$$

We used the facts that $a^\top b = b^\top a$ (line 3), that $\nabla_x(bx)^\top = \nabla_x b^\top x = b$ (line 4), and that $\nabla_xx^\top Ax = 2Ax$ for a symetric matrix $A$ (line 4).

### **Normal Equations**
Setting the above derivative to zero, we obtain the normal equations:
$$(X^\top X)\theta = X^\top y$$
Hence, the value $\theta^*$ that minimizes this objective is given by:
$$\theta^* = (X^\top X)^{-1}X^\top y.$$
Note that, we assumed that the matrix $X^\top X$ is invertible; if this is not the case, there are easy ways of addressing this issue.

Let's apply the normal equations

In [ ]:
import numpy as np

theta_best = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)
theta_best_df = pd.DataFrame(data=theta_best[np.newaxis,:], columns=X.columns)
theta_best_df

We can now use our estimate of theta to compute predictions for 3 new data points.

In [ ]:
# Collect 3 new data points
X_test = X.iloc[:3]
y_test = y.iloc[:3]

# Generate predictions on the new patients
y_test_pred = X_test.dot(theta_best)

Let's visualize the predictions

In [ ]:
# visualize the result
plt.xlabel('Body Mass Index (BMI)')
plt.ylabel('Diabetes Risk')
plt.scatter(X_train.loc[:,['bmi']], y_train)
plt.scatter(X_test.loc[:,['bmi']], y_test, color='red', marker='o')
plt.plot(X_test.loc[:,['bmi']], y_test_pred, 'x', color='red', mew=3, markersize=8)
plt.legend([ 'Initial Patients', 'New Patients', 'Predictions'])

### **Algorithm: Ordinary Least Squares**


*   **Type**: Supervised Learning (regression)
*   **Model Family**: Linear Models
*   **Objective function**: Mean Squared Error
*   **Optimization**: Normal Equations





## **Part 4: Non-Linear Least Squares**

So far, we have learned about a very simple linear model. These can only capture the simple linear relationships in the data. How can we use what we learned so far to model more complex relationships? \\
We will now see a simple approach to model complex non-linear relatioships called *least squared*

### **Recall: Polynomial functions**
Recall that a polynomial function of degree $p$ is a function of the form:
$$a_px^p + a_{p-1}x^{p-1} + \dots + a_1x^1 + a_0$$

Below are some examples of polynomial functions 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.figure(figsize=(16,4))
x_vars = np.linspace(-2,2)

plt.subplot(131)
plt.title("Quadratic Function")
plt.plot(x_vars, x_vars**2)
plt.legend(['$x^2$'])

plt.subplot(132)
plt.title("Cubic Function")
plt.plot(x_vars, x_vars**3)
plt.legend(['$x^3$'])

plt.subplot(133)
plt.title("Third degree polynomial")
plt.plot(x_vars,x_vars**3 + 2*x_vars**2 + x_vars + 1 )
plt.legend(['$x^3 + 2x^2 + x + 1$'])

### **Modeling Non-Linear Relationships With Polynomial Regression**

Specifically, given a one-dimensional continuous variable $x$, we can defining the feature function $\phi : \mathbb{R} \to \mathbb{R}^{p+1}$ as:
$$\phi(x) = \begin{bmatrix}
1 \\
x \\
x^2\\
\vdots \\
x^p
\end{bmatrix}.$$

The class of the model of form:
$$f_{\theta}(x) := \sum_{j=0}^p\theta_px_p = \theta^\top \phi(x)$$

with parameters $\theta$ and polynomial feature $\phi$ is the set of $p$-degree polynomials.

*   This model is non-linear in the input variable $x$, meaning that we can model complex data relationships.
*   It is a linear model as a function of the parameters $\theta$, meaning that we can use ordinary least squares algorithm to learn these features.  



### **The UCI Diabetes Dataset**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 4]

import numpy as np
import pandas as pd
from sklearn import datasets

# Load the dataset
X, y = datasets.load_diabetes(return_X_y = True, as_frame=True)

# Add an extra column of ones
X['ones'] = 1

# Collect 20 data points
X_train = X.iloc[-20:]
y_train = y.iloc[-20:]

# Visualize the datapoint
plt.scatter(X_train.loc[:,['bmi']], y_train, color='black')
plt.xlabel('Body Mass Index (BMI)')
plt.ylabel('Diabetes Risk')

### **Diabetes Dataset: A Non-Linear Featurization**
Let's now obtain non-linear feature for this dataset. 

In [ ]:
X_bmi = X_train.loc[:,['bmi']]

X_bmi_p3 = pd.concat([X_bmi, X_bmi**2, X_bmi**3], axis = 1)
X_bmi_p3.columns = ['bmi', 'bmi2', 'bmi3']
X_bmi_p3['ones'] = 1
X_bmi_p3.head()

In [ ]:
X_bmi_p3.T.shape

In [ ]:
y_train.shape

In [ ]:
np.linalg.inv(X_bmi_p3.T.dot(X_bmi_p3)).dot(X_bmi_p3.T).dot(y_train)

### **Diabetes Dataset: A Polynomial Model**
By training a linear model on this featurization of this diabetes dataset, we can obtain a polynomial model of diabetes risk as a function of BMI.

In [ ]:
# Fit a linear model
theta = np.linalg.inv(X_bmi_p3.T.dot(X_bmi_p3)).dot(X_bmi_p3.T).dot(y_train)

# Show the polynomial curve
x_line = np.linspace(-0.1,0.1,10)
x_line_p3 = np.stack([x_line, x_line**2, x_line**3, np.ones(10,)], axis = 1)
y_train_pred = x_line_p3.dot(theta)

plt.xlabel('Body Mass Index (BMI)')
plt.ylabel('Diabetes Risk')
plt.scatter(X_bmi, y_train)
plt.plot(x_line, y_train_pred)

### **Multivariate Polynomial Regression**
We can also take this approach to constructs non-linear functions of multiple variabels by using multivariate polynomial.\
For example, a polynomial of degree $p$ over two variables $x_1, x_2$ is a function of form:
$$a_{20}x_1^2 + a_{10}x_1 + a_{02}x_2^2 + a_{01}x_2 + a_{11}x_1x_2 + a_{00}$$

In general, a polynomial of degree $p$ over two variables $x_1, x_2$ is the function of the form:

$$f(x_1, x_2) = \sum_{i,j \geq 0, i + j \leq p}a_{ij}x_1^ix_2^j$$

In our two-dimensional example, this correspond to a feature function $\phi : \mathbb{R}^2 \to \mathbb{R}^p$ of the form 
$$\phi(x) = \begin{bmatrix}
1 \\
x_1 \\
x_1^2 \\
x_2 \\
x_2^2 \\
x_1x_2
\end{bmatrix}.$$

The same approach to holds for polynomial of an degree and any number of variables.

### **Towards General Non-Linear Features**
Any non-linear feature map $\phi(x): \mathbb{R}^d \to \mathbb{R}^p$ can be used in this way to general model of the form:
$$f_{\theta}(x) := \theta^\top \phi(x)$$

that is highly non-linear in $x$ but linear in $\theta$.

\
For example, here is a way of modeling complex periodic function via a sum of sines and cosines

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.figure(figsize=(16,4))
x_vars = np.linspace(-5,5)

plt.subplot(131)
plt.title('Cosine function')
plt.plot(x_vars, np.cos(x_vars))
plt.legend(['Cos(x)'])

plt.subplot(132)
plt.title('Sine function')
plt.plot(x_vars, np.sin(x_vars))
plt.legend(['Sin(x)'])

plt.subplot(133)
plt.title('Combination of Sine and Cosine function')
plt.plot(x_vars, np.cos(x_vars) + np.sin(2*x_vars) + np.cos(4*x_vars))
plt.legend(['Cos(x) + Sin(2x) + Cos(4x)'])

### **Algorithm: Non-Linear Least Squares**


*   Type: Supervised Learning (Regression),
*   Model family: Linear in the parameters, non-linear with respect to the raw inputs,
*   Features: Non-linear functions of the attributes
*   Objective function: Mean Squared Error
*   Optimizer: Normal Equations


